# Naive Bayes Text Classifier

### Author: Mark Kim

#### NOTE: All the code in this notebook is COMPLETE and fully functioning

Recall that the Naïve Bayes Text Classifier with Laplace Smoothing uses the
following MLE:
$$ \hat{P}(w|c) = \frac{\operatorname{count}(w,c) + 1}{\operatorname{count}(c) + \lvert
V \rvert}. $$
Once all the conditional probabilities are calculated, class labels are
determined by finding the class with the highest probability, which can be
generalized by:
$$ C_{NB} = \underset{c\in C}{\operatorname*{argmax}}\ P(c_j)\prod_{x\in X}
P(x|c) $$

This project consists of a data set of 50 restaurant reviews pulled from Yelp!
40 reviews were used for the training set and 10 were used for the test set.

## 

## Read training csv file into a data structure
What data structure do you think you should use?  Try to answer this before
seeing my approach

<details>
  <summary>Click me to see my approach</summary>
  
  #### List of Tuples
  The csv file has the text (document) being used in one column and I used an
  integer value as a flag for the class.  In the case of "negative" instances, I used 0
  and for "positive" instances, I used 1.

  Because we are dealing with different data types (a string and an integer),
  using a tuple in this case makes sense.  Likewise, the tuple will allow you to
  match indices with classes, which we will see is a benefit later on.

  So we will build a list of these tuples to store all the raw data of our corpus.
</details>

In [36]:
# Create list of tuples from csv
def listify_csv(csv_filename):
    with open(csv_filename) as f:
        # create a list of each line
        lines = f.readlines()
        # remember to test your code as you go along (e.g. print lines to see that
        # the operation functions as expected)
        # print(lines)

    # list comprehension in python (strip removes new line characters)
    lst = [(txt, int(val)) for txt,val in (tuple(line.strip().split(',')) for line in lines)]
    # the above lines are equivalent to:
    # lst = []
    # for line in lines:
    #     line_array = line.strip().split(',')
    #     line_tuple = (line_array[0], int(line_array[1]))
    #     lst.append(line_tuple)

    return lst

In [37]:
lst = listify_csv('train.csv')
lst

ValueError: too many values to unpack (expected 2)

## Build your Maximum Likelihood Estimator (MLE) and train it

Recall the MLE above.  To make the calculation of this number more readable and
simpler to understand, we may want to break up our data into multiple pieces of
data.  What pieces of data do you think we should store?  Again, like the above,
try to answer this before you see my approach.

<details>
  <summary>Click me to see my approach</summary>
  
  #### Naïve Bayes Text Classifier with Laplace Smoothing MLE
  This MLE requires three key pieces of information: the total word count of
  each class, the count of each unique word in each class, and the total number
  of unique words in the corpus (e.g. the number of words in the entire
  vocabulary).

  Since we will also eventually need the probability of each class, which is simply the
  number of documents of a particular class divided by the total number of
  documents, we might as well create another data structure that stores this
  information as well (because we will be iterating through the list we created
  above anyways).

  I think this will be best done by creating three different dictionaries.  The
  first will be a dictionary with a count of documents (value) in each class (key).  We can
  then calculate the total number of documents from this dictionary, so we
  won't need another data structure to calculate the probability of each class.

  The second dictionary will contain each unique word (key) and the counts those
  words in each class (value).  Notice that the value will need to contain
  multiple values as well (class and word count in that class).  We can embed
  another dictionary as the value (a dictionary of dictionaries).  This
  dictionary will contain all the information we need to calculate all the
  conditional probabilities.

  I initially thought that we would only need the above two dictionaries, but I
  failed to realize that I would need the count of words in
  each class to calculate the conditional probabilities.  So I added yet another
  dictionary to capture these counts.
</details>

In [34]:
# Initialize class counts.
class_counts = {}

# Initialize empty vocab count dictionary
vocab_counts = {}

# Initialize empty class word count dictionary
class_word_counts = {}

# Testing my normalization (again: test, test, test!)
# print(lst[0][0].lower().strip('.,\n').split())
doc_list = []

# Iterate through list to populate our dictionaries
for k,v in lst:
    # Populate class counts
    if v not in class_counts.keys():
        class_counts[v] = 1
    else:
        class_counts[v] += 1

    # Populate word counts for each class
    # This takes the string and splits it into a list of words while stripping punctuation
    doc_array = k.lower().strip('.,!?\n').split()
    doc_list.append(doc_array)
    # print(f'{doc_array}, {v}')

    # This iterates through the list of words to count the instances of each word
    for word in doc_array:
        # v is the integer representing the document class
        # The following code checks if the class exists in the class_word_count
        # dictionary and adds a new class if it does not, and increments the
        # count if it does
        if v not in class_word_counts.keys():
            class_word_counts[v] = 1
        else:
            class_word_counts[v] += 1
            
        # If the word in the list does not exist in the vocab_counts dictionary,
        # it adds a new entry for its document class
        if word not in vocab_counts.keys():
            vocab_counts[word] = {v: 1}
        else:
            # If the word in the list does exist in the dictionary, it must check to
            # see if the word exists in the document class.  If the document class
            # has a count, it increments the count, otherwise it adds a
            # new entry for the class where no count exists.
            if v in vocab_counts[word].keys():
                vocab_counts[word][v] += 1
            else:
                vocab_counts[word][v] = 1

In [35]:
doc_list

[['wow', 'crust', 'is', 'not', 'good'],
 ['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty'],
 ['now',
  'i',
  'am',
  'getting',
  'angry',
  'and',
  'i',
  'want',
  'my',
  'damn',
  'pho'],
 ['honeslty', 'it', "didn't", 'taste', 'that', 'fresh'],
 ['the',
  'potatoes',
  'were',
  'like',
  'rubber',
  'and',
  'you',
  'could',
  'tell',
  'they',
  'had',
  'been',
  'made',
  'up',
  'ahead',
  'of',
  'time',
  'being',
  'kept',
  'under',
  'a',
  'warmer'],
 ['would', 'not', 'go', 'back'],
 ['the',
  'cashier',
  'had',
  'no',
  'care',
  'what',
  'so',
  'ever',
  'on',
  'what',
  'i',
  'had',
  'to',
  'say',
  'it',
  'still',
  'ended',
  'up',
  'being',
  'wayyy',
  'overpriced'],
 ['i',
  'was',
  'disgusted',
  'because',
  'i',
  'was',
  'pretty',
  'sure',
  'that',
  'was',
  'human',
  'hair'],
 ['i', 'was', 'shocked', 'because', 'no', 'signs', 'indicate', 'cash', 'only'],
 ['waitress', 'was', 'a', 'little', 'slow', 'in', 'service'],
 ['did',

### Test!

The following code is to check our data to ensure that it is counting correctly.

In [4]:
class_counts

{0: 20, 1: 20}

In [5]:
class_word_counts

{0: 178, 1: 145}

In [6]:
vocab_counts

{'wow': {0: 1},
 'crust': {0: 1},
 'is': {0: 1, 1: 3},
 'not': {0: 4, 1: 1},
 'good': {0: 1, 1: 3},
 'tasty': {0: 1},
 'and': {0: 4, 1: 7},
 'the': {0: 12, 1: 8},
 'texture': {0: 1},
 'was': {0: 8, 1: 5},
 'just': {0: 1},
 'nasty': {0: 1},
 'now': {0: 1},
 'i': {0: 6, 1: 2},
 'am': {0: 1},
 'getting': {0: 1},
 'angry': {0: 1},
 'want': {0: 1},
 'my': {0: 1, 1: 2},
 'damn': {0: 1},
 'pho': {0: 1},
 'honeslty': {0: 1},
 'it': {0: 2, 1: 1},
 "didn't": {0: 1},
 'taste': {0: 2},
 'that': {0: 2},
 'fresh': {0: 1},
 'potatoes': {0: 1},
 'were': {0: 2, 1: 2},
 'like': {0: 4},
 'rubber': {0: 1},
 'you': {0: 1},
 'could': {0: 1, 1: 1},
 'tell': {0: 1},
 'they': {0: 1, 1: 2},
 'had': {0: 3},
 'been': {0: 1},
 'made': {0: 2, 1: 1},
 'up': {0: 2},
 'ahead': {0: 1},
 'of': {0: 2, 1: 1},
 'time': {0: 1},
 'being': {0: 2},
 'kept': {0: 1},
 'under': {0: 1},
 'a': {0: 4, 1: 3},
 'warmer': {0: 1},
 'would': {0: 1},
 'go': {0: 2},
 'back': {0: 1},
 'cashier': {0: 1},
 'no': {0: 2},
 'care': {0: 1},
 'wha

### LaPlace Smoothing

Recall that LaPlace smoothing increases the count of words for each class by 1.
How would you approach this problem?

<details>
  <summary>Click me to see my approach</summary>
  
  #### Simply modify the existing dictionary
  For this step, I just simply iterate through the ```vocab_counts``` dictionary
  and add a value of 1 for classes that do not have a particular word and
  increment all other entries by one.
</details>

In [7]:
for k,v in vocab_counts.items():
    for cl in class_counts.keys():
        if cl not in v.keys():
            v[cl] = 1
        else:
            v[cl] += 1

In [8]:
vocab_counts # numerator in MLE

{'wow': {0: 2, 1: 1},
 'crust': {0: 2, 1: 1},
 'is': {0: 2, 1: 4},
 'not': {0: 5, 1: 2},
 'good': {0: 2, 1: 4},
 'tasty': {0: 2, 1: 1},
 'and': {0: 5, 1: 8},
 'the': {0: 13, 1: 9},
 'texture': {0: 2, 1: 1},
 'was': {0: 9, 1: 6},
 'just': {0: 2, 1: 1},
 'nasty': {0: 2, 1: 1},
 'now': {0: 2, 1: 1},
 'i': {0: 7, 1: 3},
 'am': {0: 2, 1: 1},
 'getting': {0: 2, 1: 1},
 'angry': {0: 2, 1: 1},
 'want': {0: 2, 1: 1},
 'my': {0: 2, 1: 3},
 'damn': {0: 2, 1: 1},
 'pho': {0: 2, 1: 1},
 'honeslty': {0: 2, 1: 1},
 'it': {0: 3, 1: 2},
 "didn't": {0: 2, 1: 1},
 'taste': {0: 3, 1: 1},
 'that': {0: 3, 1: 1},
 'fresh': {0: 2, 1: 1},
 'potatoes': {0: 2, 1: 1},
 'were': {0: 3, 1: 3},
 'like': {0: 5, 1: 1},
 'rubber': {0: 2, 1: 1},
 'you': {0: 2, 1: 1},
 'could': {0: 2, 1: 2},
 'tell': {0: 2, 1: 1},
 'they': {0: 2, 1: 3},
 'had': {0: 4, 1: 1},
 'been': {0: 2, 1: 1},
 'made': {0: 3, 1: 2},
 'up': {0: 3, 1: 1},
 'ahead': {0: 2, 1: 1},
 'of': {0: 3, 1: 2},
 'time': {0: 2, 1: 1},
 'being': {0: 3, 1: 1},
 'kept'

### Populate the conditional probabilities data structure

Up to now, we have only been getting counts of words, but now we need actual
probabilities for our classifier to work.  Now that we have all the data we need
to calculate these probabilities, how will you store this information (e.g. what
would be an appropriate data structure)?

<details>
  <summary>Click me to see my approach</summary>
  
  #### Yet another dictionary!
  For this case, I will use a dictionary with an identical format as our
  ```vocab_counts``` dictionary, but instead of storing counts, I will be
  storing the conditional probabilities.
</details>

In [11]:
# Initialize an empty dictionary for the conditional probabilities
cond_prob = {}

# Iterate through our vocab_counts dictionary
for k,v in vocab_counts.items():
    # For each word, initialize a dictionary that will contain the probability
    # of the word, given a certain class.
    cond_prob[k] = {}
    for cl in v.keys():
        # The following is populated from the MLE that is seen at the beginning
        # of this notebook.
        cond_prob[k][cl] = vocab_counts[k][cl]/(class_word_counts[cl] + len(vocab_counts.keys()))

In [10]:
cond_prob

{'wow': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'crust': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'is': {0: 0.005361930294906166, 1: 0.011764705882352941},
 'not': {0: 0.013404825737265416, 1: 0.0058823529411764705},
 'good': {0: 0.005361930294906166, 1: 0.011764705882352941},
 'tasty': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'and': {0: 0.013404825737265416, 1: 0.023529411764705882},
 'the': {0: 0.03485254691689008, 1: 0.026470588235294117},
 'texture': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'was': {0: 0.024128686327077747, 1: 0.01764705882352941},
 'just': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'nasty': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'now': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'i': {0: 0.01876675603217158, 1: 0.008823529411764706},
 'am': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'getting': {0: 0.005361930294906166, 1: 0.0029411764705882353},
 'angry': {0: 0.00536

## Build our text classifier

Now that we have the conditional probabilities, we almost have a fully
functioning text classifier.  We have a bit more math to implement for the
classifier to work.  Recall from the slides what we need to do to predict the
class from a given document.
$$ C_{NB} = \underset{c\in C}{\operatorname*{argmax}}\ P(c_j)\prod_{x\in X}
P(x|c) $$
How will you approach this problem?

<details>
  <summary>Click me to see my approach</summary>
  
  #### Find the class that has the highest probability
We have the dictionary that is associated with $P(x|c)$: ```cond_prob```.  We
also can find $P(c_j)$ by using the data in ```class_counts``` as follows:
```class_counts```/```sum(class_counts.values())```.

Finally, we need to iterate through the document to get a count of all the words
so that we can calculate the probability for each class and return the class
with the greatest probability.
</details>

In [17]:
# Get test data from test.csv
test_list = listify_csv('test.csv')
test_list

[('Server did a great job handling our large rowdy table.', 1),
 ('Would come back again if I had a sushi craving while in Vegas.', 1),
 ('He deserves 5 stars.', 1),
 ('My boyfriend and I came here for the first time on a recent trip to Vegas and could not have been more pleased with the quality of food and service.',
  1),
 ('They have great dinners.', 1),
 ('Not my thing.', 0),
 ("If you are reading this please don't go there.", 0),
 ('Tonight I had the Elk Filet special...and it sucked.', 0),
 ('We ordered some old classics and some new dishes after going there a few times and were sorely disappointed with everything.',
  0),
 ('A FLY was in my apple juice.. A FLY!!!!!!!!', 0)]

In [18]:
def predict_class(tok_string):
    cl_probabilities = {}
    total_words_in_class = sum(class_counts.values())

    for cl in class_counts.keys():
        cl_probabilities[cl] = class_counts[cl]/total_words_in_class
        for word in tok_string:
            if word in cond_prob.keys():
                cl_probabilities[cl] *= cond_prob[word][cl]

    return max(cl_probabilities, key=cl_probabilities.get)
    

In [31]:
doc_array = test_list[5][0].lower().strip('.,!\n').split()
doc_array

['not', 'my', 'thing']

In [32]:
predict_class(doc_array)

0